# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [2]:
# Install required packages
%pip install ipython-sql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql
 

Note: you may need to restart the kernel to use updated packages.


In [3]:
%sql sqlite:///sql-murder-mystery.db

In [5]:
%%sql
SELECT name
FROM sqlite_master
where type = 'table'

 * sqlite:///sql-murder-mystery.db
Done.


name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
solution
income
person



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [9]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"


In [13]:
%%sql
SELECT sql
from sqlite_master


 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"
"CREATE TABLE drivers_license ( id integer PRIMARY KEY, age integer, height integer, eye_color text, hair_color text, gender text, plate_number text, car_make text, car_model text )"
"CREATE TABLE facebook_event_checkin ( person_id integer, event_id integer, event_name text, date integer, FOREIGN KEY (person_id) REFERENCES person(id) )"
"CREATE TABLE interview ( person_id integer, transcript text, FOREIGN KEY (person_id) REFERENCES person(id) )"
"CREATE TABLE get_fit_now_member ( id text PRIMARY KEY, person_id integer, name text, membership_start_date integer, membership_status text, FOREIGN KEY (person_id) REFERENCES person(id) )"
None
"CREATE TABLE get_fit_now_check_in ( membership_id text, check_in_date integer, check_in_time integer, check_out_time integer, FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id) )"
"CREATE TABLE solution ( user integer, value text )"
"CREATE TRIGGER check_solution AFTER INSERT ON solution WHEN new.user==1 BEGIN DELETE FROM solution; INSERT INTO solution VALUES (0, CASE WHEN hex(new.value)=='4A6572656D7920426F77657273' THEN ""Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."" WHEN hex(new.value)=='4D6972616E6461205072696573746C79' THEN ""Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"" ELSE ""That's not the right person. Try again!"" END ); END"
"CREATE TABLE income (ssn CHAR PRIMARY KEY, annual_income integer)"


In [26]:
%%sql
SELECT description,city
FROM crime_scene_report
WHERE date == 20180115 AND type == 'murder' AND city == "SQL City"

 * sqlite:///sql-murder-mystery.db
Done.


description,city
"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [36]:
%%sql
SELECT name, person.id, transcript, date
FROM person
LEFT JOIN interview
  ON person.id = interview.person_id
LEFT JOIN facebook_event_checkin
  ON person.id = facebook_event_checkin.person_id
WHERE (address_street_name = 'Northwestern Dr' 
   OR address_street_name = 'Franklin Ave')
   AND date == 20180115


 * sqlite:///sql-murder-mystery.db
Done.


name,id,transcript,date
Morty Schapiro,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",20180115
Annabel Miller,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",20180115


In [46]:
%%sql
SELECT p.name
FROM person p
JOIN drivers_license d
  ON p.license_id = d.id
JOIN get_fit_now_member g
  ON p.id = g.person_id
WHERE d.plate_number LIKE '%H42W%'
  AND g.membership_status = 'gold'
  AND g.id LIKE '48Z%';


 * sqlite:///sql-murder-mystery.db
Done.


name
Jeremy Bowers



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [47]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
Done.


value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


In [48]:
%%sql
SELECT transcript
FROM interview
left join person
on interview.person_id == person.id
where name=='Jeremy Bowers'

 * sqlite:///sql-murder-mystery.db
Done.


transcript
"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [49]:
%%sql
SELECT p.id, p.name
FROM person p
JOIN drivers_license d ON p.license_id = d.id
WHERE d.hair_color = 'red'
  AND d.height BETWEEN 65 AND 67
  AND d.car_make = 'Tesla'
  AND d.car_model = 'Model S';


 * sqlite:///sql-murder-mystery.db
Done.


id,name
78881,Red Korb
90700,Regina George
99716,Miranda Priestly


In [52]:
%%sql
SELECT f.person_id, COUNT(*) AS times_attended
FROM facebook_event_checkin f
WHERE f.event_name = 'SQL Symphony Concert'
  AND f.date BETWEEN 20171201 AND 20171231
  AND f.person_id IN (
    SELECT p.id
    FROM person p
    JOIN drivers_license d ON p.license_id = d.id
    WHERE d.hair_color = 'red'
      AND d.height BETWEEN 65 AND 67
      AND d.car_make = 'Tesla'
      AND d.car_model = 'Model S'
  )
GROUP BY f.person_id
HAVING COUNT(*) = 3;


 * sqlite:///sql-murder-mystery.db
Done.


person_id,times_attended
99716,3


In [54]:
%%sql
SELECT  p.id
    FROM person p
    JOIN drivers_license d ON p.license_id = d.id
    WHERE d.hair_color = 'red'
      AND d.height BETWEEN 65 AND 67
      AND d.car_make = 'Tesla'
      AND d.car_model = 'Model S'
   

 * sqlite:///sql-murder-mystery.db
Done.


id
78881
90700
99716


In [53]:
%%sql
INSERT INTO solution VALUES (1, 'Miranda Priestly');
SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
Done.


value
"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"


In [4]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
Done.


value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."
